In [ ]:
import gensim
import os

In [ ]:
import pandas as pd
devDs = pd.read_csv('/content/CT24_checkworthy_arabic_dev-test.tsv', sep = '\t')
testtDs = pd.read_csv('/content/CT24_checkworthy_arabic_dev.tsv', sep = '\t')
trainDs = pd.read_csv('/content/CT24_checkworthy_arabic_train.tsv', sep = '\t')
testDs = pd.read_csv('/content/CT24_checkworthy_arabic_test_gold.tsv', sep = '\t')

df = pd.concat([devDs, testtDs, trainDs], ignore_index = True)
df.head()

,tweet_id,tweet_url,tweet_text,class_label
0,1.384983e+18,http://twitter.com/user/status/138498294809351...,تم إعطاء أكثر من 950 مليون لقاح كوفيد حول العا...,Yes
1,1.396570e+18,http://twitter.com/user/status/139657017341556...,الشك.. اقطعه باليقين.. الطمع.. اهزمه بالقناعة....,No
2,1.478319e+18,http://twitter.com/user/status/147831862807646...,نرصد حالياً ارتفاعا في عدد الإصابات بفيروس كور...,Yes
3,1.350517e+18,http://twitter.com/user/status/135051727376580...,فيديو متداول عالميًا يجاوب لصلب سؤال أصحاب نظر...,Yes
4,1.478363e+18,http://twitter.com/user/status/147836346118490...,استكمال الحصول على جرعات اللقاح؛ حماية للأسرة ...,No


In [ ]:
no_indices = df[df['class_label'] == "NO"].index[:100]  # Select only the first 100 indices
df = df.drop(no_indices)
df['class_label'].value_counts()

class_label
No     5895
Yes    3031
Name: count, dtype: int64

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

story = []
for index, row in df.iterrows():
  text = row['tweet_text']
  raw_sent = sent_tokenize(text)
  for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [ ]:
word2vecModel = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [ ]:
word2vecModel.build_vocab(story)

In [ ]:
word2vecModel.train(story, total_examples = word2vecModel.corpus_count, epochs = word2vecModel.epochs)

(903585, 1233785)

In [ ]:
df['class_label'] = df['class_label'].apply(lambda x: 1 if x == 'Yes' else 0)
df2 = testDs
df2['class_label'] = df2['class_label'].apply(lambda x: 1 if x == 'Yes' else 0)

In [ ]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from keras.models import Sequential, Model
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional, GRU, MultiHeadAttention, LayerNormalization, Concatenate, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score
from nltk.tokenize import word_tokenize
from keras.callbacks import EarlyStopping
from imblearn.over_sampling import RandomOverSampler

def get_word_embedding_model(word_embedding_type):
    if word_embedding_type == "word2vec":
        word_embedding_model = word2vecModel
    else:
        raise ValueError("Unsupported word embedding type. Choose 'word2vec' or 'fasttext'.")
    return word_embedding_model

def get_sentence_embedding(sentence, model, word_embedding_type):
    if word_embedding_type == "word2vec":
        tokens = word_tokenize(sentence.lower())
        vectors = [model.wv[token] for token in tokens if token in model.wv]
        if not vectors:
            return np.zeros(model.vector_size)
        return np.mean(vectors, axis=0)
    else:
        raise ValueError("Unsupported word embedding type. Choose 'word2vec' or 'fasttext'.")

def create_embedding_matrix(word_index, word_embedding_model, embedding_dim):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        if word in word_embedding_model.wv:
            embedding_matrix[i] = word_embedding_model.wv[word]
        else:
            embedding_matrix[i] = np.zeros(embedding_dim)
    return embedding_matrix

def create_stacked_lstm_model(embedding_matrix, input_length, model_type):
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0],
                        output_dim=embedding_matrix.shape[1],
                        weights=[embedding_matrix],
                        input_length=input_length,
                        trainable=False))
    model.add(SpatialDropout1D(0.2))

    if model_type == "lstm":
        model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
        model.add(Bidirectional(LSTM(50, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
    elif model_type == "bilstm":
        model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
        model.add(Bidirectional(LSTM(50, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
    elif model_type == "gru":
        model.add(GRU(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
        model.add(GRU(50, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
    else:
        raise ValueError("Unsupported model type. Choose 'lstm', 'bilstm', or 'gru'.")

    query = model.output
    key = model.output
    value = model.output

    attention_layer = MultiHeadAttention(num_heads=4, key_dim=50)
    attention_output = attention_layer(query=query, value=value, key=key)
    attention_output = LayerNormalization()(attention_output)

    # Concatenate LSTM output and attention output
    combined_output = Concatenate()([model.output, attention_output])

    # Apply GlobalAveragePooling1D to aggregate the sequence into a single vector
    combined_output = GlobalAveragePooling1D()(combined_output)

    combined_output = Dense(1, activation='sigmoid')(combined_output)

    final_model = Model(inputs=model.input, outputs=combined_output)
    final_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return final_model

def train_and_evaluate_model(train_data, dev_data, word_embedding_type, model_type):
    y_train = train_data["class_label"].values
    y_dev = dev_data["class_label"].values

    word_embedding_model = get_word_embedding_model(word_embedding_type)

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train_data["tweet_text"].tolist() + dev_data["tweet_text"].tolist())
    X_train_sequences = tokenizer.texts_to_sequences(train_data["tweet_text"].tolist())
    X_dev_sequences = tokenizer.texts_to_sequences(dev_data["tweet_text"].tolist())

    word_index = tokenizer.word_index
    embedding_matrix = create_embedding_matrix(word_index, word_embedding_model, word_embedding_model.vector_size)

    max_sequence_length = max(len(seq) for seq in X_train_sequences + X_dev_sequences)
    X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
    X_dev_padded = pad_sequences(X_dev_sequences, maxlen=max_sequence_length)

    # Handle data imbalance using random oversampling
    ros = RandomOverSampler(random_state=42)
    X_train_padded_resampled, y_train_resampled = ros.fit_resample(X_train_padded, y_train)

    stacked_lstm_model = create_stacked_lstm_model(embedding_matrix, max_sequence_length, model_type)

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    stacked_lstm_model.fit(X_train_padded_resampled, y_train_resampled, epochs=5, batch_size=32,
                           validation_data=(X_dev_padded, y_dev),
                           callbacks=[early_stopping])

    dev_predictions = (stacked_lstm_model.predict(X_dev_padded, verbose=0) > 0.5).astype("int32").flatten()
    y_dev = y_dev.flatten()
    f1score = f1_score(y_dev, dev_predictions)

    print(f"F1-score on Development Set: {f1score}")
    return stacked_lstm_model

In [ ]:
word_embedding_type = "word2vec"
model_type = "lstm"
train_data = df
dev_data = df2

stacked_lstm_model = train_and_evaluate_model(train_data, dev_data, word_embedding_type, model_type)

Epoch 1/5
369/369 [==============================] - 215s 511ms/step - loss: 0.6754 - accuracy: 0.5817 - val_loss: 0.8789 - val_accuracy: 0.3984
Epoch 2/5
369/369 [==============================] - 174s 471ms/step - loss: 0.6548 - accuracy: 0.6132 - val_loss: 0.7037 - val_accuracy: 0.5508
Epoch 3/5
369/369 [==============================] - 176s 476ms/step - loss: 0.6477 - accuracy: 0.6216 - val_loss: 0.6924 - val_accuracy: 0.5754
Epoch 4/5
369/369 [==============================] - 175s 473ms/step - loss: 0.6461 - accuracy: 0.6197 - val_loss: 0.7051 - val_accuracy: 0.5869
Epoch 5/5
369/369 [==============================] - 174s 471ms/step - loss: 0.6431 - accuracy: 0.6244 - val_loss: 0.7596 - val_accuracy: 0.4770
F1-score on Development Set: 0.4037383177570093


In [ ]:
word_embedding_type = "word2vec"
model_type = "bilstm"
train_data = df
dev_data = df2
stacked_lstm_model = train_and_evaluate_model(train_data, dev_data, word_embedding_type, model_type)

Epoch 1/5
369/369 [==============================] - 268s 674ms/step - loss: 0.6800 - accuracy: 0.5773 - val_loss: 0.7631 - val_accuracy: 0.4672
Epoch 2/5
369/369 [==============================] - 250s 677ms/step - loss: 0.6543 - accuracy: 0.6140 - val_loss: 0.7563 - val_accuracy: 0.4852
Epoch 3/5
369/369 [==============================] - 249s 674ms/step - loss: 0.6511 - accuracy: 0.6139 - val_loss: 0.7021 - val_accuracy: 0.4902
Epoch 4/5
369/369 [==============================] - 248s 672ms/step - loss: 0.6487 - accuracy: 0.6203 - val_loss: 0.7434 - val_accuracy: 0.4885
Epoch 5/5
369/369 [==============================] - 249s 674ms/step - loss: 0.6416 - accuracy: 0.6311 - val_loss: 0.7386 - val_accuracy: 0.4770
F1-score on Development Set: 0.42933810375670844


In [ ]:
word_embedding_type = "word2vec"
model_type = "gru"
train_data = df
dev_data = df2
stacked_lstm_model = train_and_evaluate_model(train_data, dev_data, word_embedding_type, model_type)

Epoch 1/5
369/369 [==============================] - 131s 329ms/step - loss: 0.6733 - accuracy: 0.5796 - val_loss: 0.7192 - val_accuracy: 0.4787
Epoch 2/5
369/369 [==============================] - 119s 321ms/step - loss: 0.6559 - accuracy: 0.6050 - val_loss: 0.7595 - val_accuracy: 0.4541
Epoch 3/5
369/369 [==============================] - 126s 342ms/step - loss: 0.6464 - accuracy: 0.6225 - val_loss: 0.7305 - val_accuracy: 0.4951
Epoch 4/5
369/369 [==============================] - 120s 326ms/step - loss: 0.6411 - accuracy: 0.6232 - val_loss: 0.7232 - val_accuracy: 0.5180
F1-score on Development Set: 0.40892193308550184
